In [1]:
import os
import time
# os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
# os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask
# import modin.pandas as pd
import pandas as pd
import numpy as np
# import FeatureUtils
from sklearn.preprocessing import OneHotEncoder
import torch
import train_utils as tu
from sklearn.model_selection import train_test_split
import itertools


In [2]:
data = pd.read_csv('base01.csv')
del data['Unnamed: 0']

In [3]:
df = data.copy()

In [4]:
data = data.head(1000)

In [5]:
def exf(row, group_name, s):
    return data[eval(s)]['diff_time'].count()

def data_finder(combine_list, group_list, eq_list):
    index = 1
#     print(1)
    for group_name in group_list:
        for combines in combine_list:
            for eq in eq_list:
                s = ""
                s += '(data[group_name] == row[group_name])'
                for i,combine in enumerate(combines):
#                     s += combine[0]+eq[i]+combine[1]+'     '
                    s +=  '&(row["' + combine[0] + '"]' + eq[i] + 'data["' + combine[1] + '"])'
                data['_'+str(index)] = data.apply(lambda row: exf(row, group_name, s), axis=1)
                print(index,s)
                index = index+1

In [6]:
combine_list = ['QUEUE_START_TIME', 'ENTRY_TIME', 'ENTRY_NOTICE_TIME']
clist = itertools.product(combine_list,combine_list)
group_list = ['MAT_CODE_label']
clist = itertools.combinations_with_replacement(clist,2)
clist = list(clist)
group_list = list(group_list)
eq = itertools.product('<>','<>')
# eq = itertools.product(eq,'<>')
a = [ "".join(list(i)) for i in eq]
print(a)


['<<', '<>', '><', '>>']


In [7]:
data_finder(clist,group_list,a)

1 (data[group_name] == row[group_name])&(row["QUEUE_START_TIME"]<data["QUEUE_START_TIME"])&(row["QUEUE_START_TIME"]<data["QUEUE_START_TIME"])
2 (data[group_name] == row[group_name])&(row["QUEUE_START_TIME"]<data["QUEUE_START_TIME"])&(row["QUEUE_START_TIME"]>data["QUEUE_START_TIME"])
3 (data[group_name] == row[group_name])&(row["QUEUE_START_TIME"]>data["QUEUE_START_TIME"])&(row["QUEUE_START_TIME"]<data["QUEUE_START_TIME"])
4 (data[group_name] == row[group_name])&(row["QUEUE_START_TIME"]>data["QUEUE_START_TIME"])&(row["QUEUE_START_TIME"]>data["QUEUE_START_TIME"])
5 (data[group_name] == row[group_name])&(row["QUEUE_START_TIME"]<data["QUEUE_START_TIME"])&(row["QUEUE_START_TIME"]<data["ENTRY_TIME"])
6 (data[group_name] == row[group_name])&(row["QUEUE_START_TIME"]<data["QUEUE_START_TIME"])&(row["QUEUE_START_TIME"]>data["ENTRY_TIME"])
7 (data[group_name] == row[group_name])&(row["QUEUE_START_TIME"]>data["QUEUE_START_TIME"])&(row["QUEUE_START_TIME"]<data["ENTRY_TIME"])
8 (data[group_name] == r

61 (data[group_name] == row[group_name])&(row["QUEUE_START_TIME"]<data["ENTRY_TIME"])&(row["ENTRY_NOTICE_TIME"]<data["ENTRY_TIME"])
62 (data[group_name] == row[group_name])&(row["QUEUE_START_TIME"]<data["ENTRY_TIME"])&(row["ENTRY_NOTICE_TIME"]>data["ENTRY_TIME"])
63 (data[group_name] == row[group_name])&(row["QUEUE_START_TIME"]>data["ENTRY_TIME"])&(row["ENTRY_NOTICE_TIME"]<data["ENTRY_TIME"])
64 (data[group_name] == row[group_name])&(row["QUEUE_START_TIME"]>data["ENTRY_TIME"])&(row["ENTRY_NOTICE_TIME"]>data["ENTRY_TIME"])
65 (data[group_name] == row[group_name])&(row["QUEUE_START_TIME"]<data["ENTRY_TIME"])&(row["ENTRY_NOTICE_TIME"]<data["ENTRY_NOTICE_TIME"])
66 (data[group_name] == row[group_name])&(row["QUEUE_START_TIME"]<data["ENTRY_TIME"])&(row["ENTRY_NOTICE_TIME"]>data["ENTRY_NOTICE_TIME"])
67 (data[group_name] == row[group_name])&(row["QUEUE_START_TIME"]>data["ENTRY_TIME"])&(row["ENTRY_NOTICE_TIME"]<data["ENTRY_NOTICE_TIME"])
68 (data[group_name] == row[group_name])&(row["QUEUE_ST

121 (data[group_name] == row[group_name])&(row["ENTRY_TIME"]<data["ENTRY_TIME"])&(row["ENTRY_TIME"]<data["ENTRY_TIME"])
122 (data[group_name] == row[group_name])&(row["ENTRY_TIME"]<data["ENTRY_TIME"])&(row["ENTRY_TIME"]>data["ENTRY_TIME"])
123 (data[group_name] == row[group_name])&(row["ENTRY_TIME"]>data["ENTRY_TIME"])&(row["ENTRY_TIME"]<data["ENTRY_TIME"])
124 (data[group_name] == row[group_name])&(row["ENTRY_TIME"]>data["ENTRY_TIME"])&(row["ENTRY_TIME"]>data["ENTRY_TIME"])
125 (data[group_name] == row[group_name])&(row["ENTRY_TIME"]<data["ENTRY_TIME"])&(row["ENTRY_TIME"]<data["ENTRY_NOTICE_TIME"])
126 (data[group_name] == row[group_name])&(row["ENTRY_TIME"]<data["ENTRY_TIME"])&(row["ENTRY_TIME"]>data["ENTRY_NOTICE_TIME"])
127 (data[group_name] == row[group_name])&(row["ENTRY_TIME"]>data["ENTRY_TIME"])&(row["ENTRY_TIME"]<data["ENTRY_NOTICE_TIME"])
128 (data[group_name] == row[group_name])&(row["ENTRY_TIME"]>data["ENTRY_TIME"])&(row["ENTRY_TIME"]>data["ENTRY_NOTICE_TIME"])
129 (data[gr

In [8]:
data = data.fillna(0)

In [9]:
import scipy as sc
import math
tmp = [ (sc.stats.pearsonr( data['MAT_CODE_label'],data['_'+str(i+1)])[0], i+1) for i in range(180)]
# tmp = tmp.sort()
tmp = [i for i in tmp if not math.isnan(i[0])]
tmp
sorted(tmp)

/Users/mac/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3399: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


[(-0.1921479370551042, 91),
 (-0.1843936616471048, 147),
 (-0.039450906317933625, 79),
 (-0.03454599237924184, 131),
 (-0.020014255862041985, 150),
 (0.06266876066985025, 111),
 (0.06469777788384029, 42),
 (0.13405417518202406, 155),
 (0.14350315401322833, 139),
 (0.2008971660970954, 174),
 (0.24419340615491952, 135),
 (0.25387203145827053, 138),
 (0.2624761457424204, 12),
 (0.2624761457424204, 72),
 (0.2624761457424204, 76),
 (0.2624761457424204, 84),
 (0.2624761457424204, 88),
 (0.2624761457424204, 96),
 (0.26338696953051066, 80),
 (0.26378159325201167, 8),
 (0.26378159325201167, 40),
 (0.26378159325201167, 44),
 (0.26378159325201167, 48),
 (0.26378159325201167, 52),
 (0.26378159325201167, 56),
 (0.26378159325201167, 60),
 (0.26378159325201167, 64),
 (0.26378159325201167, 68),
 (0.2664437188473146, 25),
 (0.2664437188473146, 57),
 (0.2664437188473146, 85),
 (0.2664437188473146, 157),
 (0.2664437188473146, 161),
 (0.2664437188473146, 165),
 (0.26694466082684565, 129),
 (0.267035554945

In [10]:
data_copy = data.copy()

In [11]:
del data['ENTRY_NOTICE_TIME']
del data['ENTRY_TIME']
del data['QUEUE_START_TIME']
for i in range(180):
    del data['_'+str(i+1)]

In [12]:
def linearNet(x_train, y_train,batch_size=None, optimizer=None, net=None):
    num_inputs = x_train.shape[1]
#     print(num_inputs)
    dataset = Data.TensorDataset(x_train, y_train)
    if batch_size is None:
        batch_size = 512
    data_iter = Data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    if net is None:
        net = nn.Sequential(
            nn.Linear(num_inputs, 1),
#             nn.ReLU()
        )
    nn.init.normal_(net[0].weight, mean=0, std=0.01)
    nn.init.constant_(net[0].bias, val=0)
    if optimizer is None:
        optimizer = torch.optim.Adam(net.parameters(), lr=0.1)
    num_epoch = 36
    loss = nn.MSELoss()
    
    for epoch in range(num_epoch):
        for x, y in data_iter:
            output = net(x)
            l = loss(output, y.view(-1, 1))
            optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
            l.backward()
            optimizer.step()
#         print('epoch %d, loss: %f' % (epoch, l.item()))
    return net

In [13]:
y = data['diff_time']
del data['diff_time']
x = data.__array__()
y = torch.tensor(np.array(y), dtype=torch.float32)
x = torch.tensor(np.array(x), dtype=torch.float32)
x.shape,y.shape

(torch.Size([1000, 1]), torch.Size([1000]))

In [14]:
from torch.utils import data as Data
from torch import nn
net = linearNet(x, y)


In [15]:
from sklearn import metrics
out = net(torch.tensor(np.array(x), dtype=torch.float32)).detach().numpy()
y_ = y.detach().numpy()
np.sqrt(metrics.mean_squared_error(y_, out))


256.27832

In [16]:
out = net(x)
y_ = y.resize_as(out)
labels = y_ - out
labels

/Users/mac/anaconda3/lib/python3.7/site-packages/torch/tensor.py:319: UserWarning: non-inplace resize_as is deprecated
  warnings.warn("non-inplace resize_as is deprecated")


tensor([[-1.3552e+02],
        [-1.3562e+02],
        [-7.5327e+01],
        [-1.3276e+02],
        [-7.6311e+01],
        [-1.3974e+02],
        [-1.3336e+02],
        [-1.1352e+02],
        [ 1.4285e+02],
        [-1.3607e+02],
        [ 1.3995e+02],
        [-1.1782e+02],
        [ 2.6111e+01],
        [ 1.3838e+02],
        [-1.3941e+02],
        [-1.6274e+02],
        [-1.6547e+02],
        [-1.2301e+02],
        [-1.4312e+02],
        [-1.2374e+02],
        [-1.4554e+02],
        [ 1.3103e+02],
        [ 1.3070e+02],
        [-1.7012e+02],
        [-1.5484e+02],
        [-1.3067e+02],
        [-1.3107e+02],
        [-1.0676e+02],
        [-1.5854e+02],
        [-1.0901e+02],
        [-1.5462e+02],
        [-1.5462e+02],
        [-1.5481e+02],
        [-1.6810e+02],
        [-1.1109e+02],
        [-1.6451e+02],
        [ 1.1918e+02],
        [-8.8539e+01],
        [ 2.0616e+02],
        [-9.0606e+01],
        [ 2.0334e+02],
        [-4.6956e+01],
        [ 2.0153e+02],
        [-1

In [ ]:
out

In [ ]:
x = data_copy['_1'].__array__()
x

In [17]:
labels = labels.detach().numpy()
labels = torch.tensor(np.array(labels), dtype=torch.float32)

In [18]:
def mean_squared_error(data_copy, s, y):
    from sklearn import metrics
    x = data_copy[s].__array__()
    x = torch.tensor(np.array(x), dtype=torch.float32)
    x = x.reshape([-1, 1])
    net = linearNet(x, y)
    net(x)
    out = net(x)
    out = out.detach().numpy()
    y_ = y.detach().numpy()
#     print(x)
    return np.sqrt(metrics.mean_squared_error(y_, out))

In [ ]:
data_copy

In [ ]:
x = torch.tensor(np.array(x), dtype=torch.float32)
y = labels.detach().numpy()
y = torch.tensor(np.array(y), dtype=torch.float32)
x = x.reshape([-1,1])
net = tu.linearNet(x, y)


In [ ]:
a = mean_squared_error(data_copy, '_5', labels)
a

In [ ]:
labels

In [ ]:
y

In [ ]:
out = net(torch.tensor(np.array(x), dtype=torch.float32)).detach().numpy()
y_ = y.detach().numpy()
np.sqrt(metrics.mean_squared_error(y_, out))

In [20]:
data_copy['_7']

0        0
1        1
2        0
3        2
4        1
5        0
6        3
7        4
8        0
9        5
10       1
11       6
12       7
13       2
14       8
15       3
16       4
17       9
18      10
19      11
20      12
21       5
22       6
23       7
24       1
25      13
26      14
27      15
28       2
29      16
      ... 
970    110
971     17
972      1
973    111
974    112
975    113
976    112
977      1
978      6
979     18
980     19
981     19
982    107
983    108
984    109
985      5
986     17
987    106
988    107
989    108
990    108
991     11
992     12
993     13
994     14
995    105
996      1
997      2
998    103
999     14
Name: _7, Length: 1000, dtype: int64

In [21]:
tmp = [ (mean_squared_error(data_copy, '_'+str(i+1), labels), i+1) for i in range(180)]
# tmp = tmp.sort()
tmp = [i for i in tmp if not math.isnan(i[0])]
tmp
sorted(tmp)

[(201.92697, 90),
 (203.80197, 62),
 (204.40318, 30),
 (205.6299, 34),
 (206.20595, 18),
 (207.82883, 78),
 (207.8464, 94),
 (211.51485, 50),
 (211.79591, 22),
 (212.36406, 66),
 (214.98595, 82),
 (218.44333, 54),
 (221.30194, 26),
 (225.93616, 14),
 (230.36526, 86),
 (231.73898, 58),
 (232.52249, 74),
 (233.78967, 46),
 (246.03688, 11),
 (247.1699, 7),
 (248.93169, 160),
 (248.95174, 112),
 (249.19064, 148),
 (249.29828, 100),
 (249.31354, 132),
 (249.38083, 128),
 (249.38951, 124),
 (249.39238, 104),
 (249.40323, 152),
 (249.40367, 176),
 (249.40556, 136),
 (249.409, 164),
 (249.41095, 116),
 (249.41248, 172),
 (249.5421, 108),
 (249.54688, 144),
 (249.78662, 140),
 (249.94876, 156),
 (249.96191, 120),
 (249.96448, 168),
 (249.9776, 180),
 (250.03099, 91),
 (250.69923, 35),
 (250.81868, 167),
 (250.9464, 23),
 (251.2434, 163),
 (251.52017, 130),
 (251.76923, 147),
 (251.86551, 146),
 (251.86607, 19),
 (251.90147, 31),
 (252.11346, 85),
 (252.11424, 161),
 (252.11493, 25),
 (252.11595

In [ ]:
data_copy['_30']

In [27]:
def exf(row, group_name, s):
    return df[eval(s)]['diff_time'].count()

In [24]:
df

,ENTRY_NOTICE_TIME,ENTRY_TIME,QUEUE_START_TIME,diff_time,MAT_CODE_label
0,2019-06-15 01:41:55,2019-06-15 01:43:11,2019-06-15 00:00:25,101.500000,44
1,2019-06-15 01:41:55,2019-06-15 01:43:20,2019-06-15 00:00:31,101.400000,44
2,2019-06-15 00:36:55,2019-06-15 00:45:28,2019-06-15 00:01:43,35.200000,20
3,2019-06-15 01:46:55,2019-06-15 01:52:06,2019-06-15 00:02:39,104.266667,44
4,2019-06-15 00:36:55,2019-06-15 00:39:52,2019-06-15 00:02:42,34.216667,20
5,2019-06-15 00:36:55,2019-06-15 00:40:03,2019-06-15 00:02:53,34.033333,32
6,2019-06-15 01:46:55,2019-06-15 01:57:15,2019-06-15 00:03:15,103.666667,44
7,2019-06-15 02:06:55,2019-06-15 02:12:46,2019-06-15 00:03:25,123.500000,44
8,2019-06-15 05:36:55,2019-06-15 05:55:22,2019-06-15 00:04:29,332.433333,35
9,2019-06-15 01:46:55,2019-06-15 02:07:32,2019-06-15 00:05:58,100.950000,44


In [29]:
s = '(df[group_name] == row[group_name])&(row["QUEUE_START_TIME"]>df["QUEUE_START_TIME"])&(row["QUEUE_START_TIME"]<df["ENTRY_TIME"])'
df['_0'] = df.apply(lambda row: exf(row, 'MAT_CODE_label',s), axis=1)


In [ ]:
df_copy = df.copy()

In [30]:
df = df.fillna(0)
df

,ENTRY_NOTICE_TIME,ENTRY_TIME,QUEUE_START_TIME,diff_time,MAT_CODE_label,_0
0,2019-06-15 01:41:55,2019-06-15 01:43:11,2019-06-15 00:00:25,101.500000,44,0
1,2019-06-15 01:41:55,2019-06-15 01:43:20,2019-06-15 00:00:31,101.400000,44,1
2,2019-06-15 00:36:55,2019-06-15 00:45:28,2019-06-15 00:01:43,35.200000,20,0
3,2019-06-15 01:46:55,2019-06-15 01:52:06,2019-06-15 00:02:39,104.266667,44,2
4,2019-06-15 00:36:55,2019-06-15 00:39:52,2019-06-15 00:02:42,34.216667,20,1
5,2019-06-15 00:36:55,2019-06-15 00:40:03,2019-06-15 00:02:53,34.033333,32,0
6,2019-06-15 01:46:55,2019-06-15 01:57:15,2019-06-15 00:03:15,103.666667,44,3
7,2019-06-15 02:06:55,2019-06-15 02:12:46,2019-06-15 00:03:25,123.500000,44,4
8,2019-06-15 05:36:55,2019-06-15 05:55:22,2019-06-15 00:04:29,332.433333,35,0
9,2019-06-15 01:46:55,2019-06-15 02:07:32,2019-06-15 00:05:58,100.950000,44,5


In [31]:
y = df['diff_time']
del df['ENTRY_NOTICE_TIME']
del df['ENTRY_TIME']
del df['QUEUE_START_TIME']
del df['diff_time']


In [32]:
x = df.__array__()
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=1)

In [33]:
net = tu.torch_linear(X_train, y_train, X_test, y_test, 'home/feature_07_1')

epoch 0, loss: 589390.875000
epoch 1, loss: 800229.562500
epoch 2, loss: 349912.375000
epoch 3, loss: 579356.062500
epoch 4, loss: 288308.531250
epoch 5, loss: 389039.375000
epoch 6, loss: 333559.187500
epoch 7, loss: 444883.500000
epoch 8, loss: 315280.781250
epoch 9, loss: 235711.000000
epoch 10, loss: 273859.312500
epoch 11, loss: 425954.281250
epoch 12, loss: 360572.312500
epoch 13, loss: 496973.187500
epoch 14, loss: 355400.312500
epoch 15, loss: 267535.375000


In [ ]:
X_train, X_test, y_train, y_test